In [1]:
#Clustering Music Genres with Machine Learning
# https://thecleverprogrammer.com/2022/04/05/clustering-music-genres-with-machine-learning/

In [2]:
#gerekli kütüphaneleri import edelim
import pandas as pd
import numpy as np
from sklearn import cluster

In [3]:

#verisetini okuyalım.
data = pd.read_csv("C:/Users/Asus/Downloads/archive/Spotify-2000.csv")
print(data.head())

   Index                   Title             Artist            Top Genre  \
0      1                 Sunrise        Norah Jones      adult standards   
1      2             Black Night        Deep Purple           album rock   
2      3          Clint Eastwood           Gorillaz  alternative hip hop   
3      4           The Pretender       Foo Fighters    alternative metal   
4      5  Waitin' On A Sunny Day  Bruce Springsteen         classic rock   

   Year  Beats Per Minute (BPM)  Energy  Danceability  Loudness (dB)  \
0  2004                     157      30            53            -14   
1  2000                     135      79            50            -11   
2  2001                     168      69            66             -9   
3  2007                     173      96            43             -4   
4  2002                     106      82            58             -5   

   Liveness  Valence Length (Duration)  Acousticness  Speechiness  Popularity  
0        11       68          

In [4]:
#işe yaramayan(kullanılmayacak)sütunları kaldıralım.
data = data.drop("Index", axis=1)

In [5]:
#şimdi korelasyon özelliklerine bakalım.
# Sayısal sütunları seçiyoruz.
numeric_data = data.select_dtypes(include=['number'])

# Korelasyon hesaplama
print(numeric_data.corr())


                            Year  Beats Per Minute (BPM)    Energy  \
Year                    1.000000                0.012570  0.147235   
Beats Per Minute (BPM)  0.012570                1.000000  0.156644   
Energy                  0.147235                0.156644  1.000000   
Danceability            0.077493               -0.140602  0.139616   
Loudness (dB)           0.343764                0.092927  0.735711   
Liveness                0.019017                0.016256  0.174118   
Valence                -0.166163                0.059653  0.405175   
Acousticness           -0.132946               -0.122472 -0.665156   
Speechiness             0.054097                0.085598  0.205865   
Popularity             -0.158962               -0.003181  0.103393   

                        Danceability  Loudness (dB)  Liveness   Valence  \
Year                        0.077493       0.343764  0.019017 -0.166163   
Beats Per Minute (BPM)     -0.140602       0.092927  0.016256  0.059653   
Ener

In [6]:
# Veriden "Beats Per Minute (BPM)", "Loudness (dB)", "Liveness", "Valence", "Acousticness" ve "Speechiness"
# sütunları seçilerek data2 adlı yeni bir veri çerçevesi oluşturuluyor.

data2 = data[["Beats Per Minute (BPM)", "Loudness (dB)", 
              "Liveness", "Valence", "Acousticness", 
              "Speechiness"]]

from sklearn.preprocessing import MinMaxScaler
for i in data.columns:
    MinMaxScaler(i)

#"Beats Per Minute (BPM)" gibi sütunları içeren verilere K-means algoritmasıyla 10 küme belirlemek için KMeans nesnesi oluşturuluyor.    
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10)
#Beats Per Minute (BPM)" ve diğer sütunlar kullanılarak kümeler tahmin ediliyor ve sonuçlar clusters değişkenine atanıyor.
clusters = kmeans.fit_predict(data2)

In [7]:
#Kümeler,veri çerçevesine "Music Segments" sütunu olarak eklendikten sonra,kümeler etiketlerle yeniden adlandırılmıştır.
data["Music Segments"] = clusters
MinMaxScaler(data["Music Segments"])
data["Music Segments"] = data["Music Segments"].map({1: "Cluster 1", 2: 
    "Cluster 2", 3: "Cluster 3", 4: "Cluster 4", 5: "Cluster 5", 
    6: "Cluster 6", 7: "Cluster 7", 8: "Cluster 8", 
    9: "Cluster 9", 10: "Cluster 10"})

In [8]:
#veri setinin ilk 5 satırını görelim.
print(data.head())

                    Title             Artist            Top Genre  Year  \
0                 Sunrise        Norah Jones      adult standards  2004   
1             Black Night        Deep Purple           album rock  2000   
2          Clint Eastwood           Gorillaz  alternative hip hop  2001   
3           The Pretender       Foo Fighters    alternative metal  2007   
4  Waitin' On A Sunny Day  Bruce Springsteen         classic rock  2002   

   Beats Per Minute (BPM)  Energy  Danceability  Loudness (dB)  Liveness  \
0                     157      30            53            -14        11   
1                     135      79            50            -11        17   
2                     168      69            66             -9         7   
3                     173      96            43             -4         3   
4                     106      82            58             -5        10   

   Valence Length (Duration)  Acousticness  Speechiness  Popularity  \
0       68           

In [9]:
import plotly.graph_objects as go
PLOT = go.Figure()
#Veri kümesindeki benzersiz "Music Segments" değerleri üzerinde döngü başlatılıyor.
for i in list(data["Music Segments"].unique()):
    
#Her küme için "Beats Per Minute (BPM)", "Energy" ve "Danceability" değerleri bir 3D scatter grafiğine ekleniyor.
    PLOT.add_trace(go.Scatter3d(x = data[data["Music Segments"]== i]['Beats Per Minute (BPM)'],
                                y = data[data["Music Segments"] == i]['Energy'],
                                z = data[data["Music Segments"] == i]['Danceability'],                        
                                mode = 'markers',marker_size = 6, marker_line_width = 1,
                                name = str(i)))
PLOT.update_traces(hovertemplate='Beats Per Minute (BPM): %{x} <br>Energy: %{y} <br>Danceability: %{z}')

#Marker ayarları (boyut, çizgi genişliği) ve küme adı belirleniyor.    
PLOT.update_layout(width = 800, height = 800, autosize = True, showlegend = True,
                   
#Hover (fareyle üzerine gelme) bilgisi için x, y ve z değerleri ekleniyor.
                   scene = dict(xaxis=dict(title = 'Beats Per Minute (BPM)', titlefont_color = 'black'),
                                yaxis=dict(title = 'Energy', titlefont_color = 'black'),
                                zaxis=dict(title = 'Danceability', titlefont_color = 'black')),
                                
#Grafik düzeni; eksen başlıkları, yazı tipi, boyut ve legend gibi ayarlarla özelleştiriliyor.                                
                   font = dict(family = "Gilroy", color  = 'black', size = 12))